In [ ]:
from datetime import datetime
import numpy as np
import alpaca_trade_api as tradeapi
import pandas as pd
import threading
from time import sleep 
import time as time
import json
import logging
import datetime


APCA_API_KEY_ID = 'PKTVWYF8T2T1UEUJVA5T'
APCA_API_SECRET_KEY = '2euAseS6hCblaZQRAE0odvz3/zuLttrz85ba8vyr'
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'

class SMAStrat:
    def __init__(self):
        self.alpaca = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, api_version='v2')
        self.barTimeframe = "day" # 1Min, 5Min, 15Min, 1H, 1D
        self.assetsToTrade = ["SPY","MSFT","AAPL","NFLX","MRK","PEP","DG","ADP"]
        self.positionSizing = 1/len(self.assetsToTrade)
        self.timeList = []
        self.openList = []
        self.highList = []
        self.lowList = []
        self.closeList = []
        self.volumeList = []
    
    def run(self):
        # First, cancel any existing orders so they don't impact our buying power.
        orders = self.alpaca.list_orders(status="open")
        for order in orders:
          self.alpaca.cancel_order(order.id)

        # Wait for market to open.
        print("Waiting for market to open...")
        tAMO = threading.Thread(target=self.awaitMarketOpen)
        tAMO.start()
        tAMO.join()
        print("Market opened.")
        while True:

          # Figure out when the market will close so we can prepare to sell beforehand.
          clock = self.alpaca.get_clock()
          closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          self.timeToClose = closingTime - currTime

          if(self.timeToClose < (60 * 5)):
            # Stop Trading when 5 minutes til market close.
            print("Market closing soon. No Longer Trading.")
            # Run script again after market close for next trading day.
            print("Sleeping until market close (5 minutes).")
            time.sleep(60 * 15)
          else:
            # Rebalance the portfolio.
            for symbol in self.assetsToTrade:
                self.dataset = self.alpaca.get_barset(symbol,self.barTimeframe,limit=100).df
                tSMA = threading.Thread(target=self.calculateSMAs(symbol))
                tSMA.start()
                tSMA.join()
                tRebalance = threading.Thread(target=self.buyselldecision(symbol))
                tRebalance.start()
                tRebalance.join()
            time.sleep(60*10) ###This determines how frequently it runs. a value of 60 would mean every minute
        
  # Wait for market to open.
    def awaitMarketOpen(self):
        isOpen = self.alpaca.get_clock().is_open
        while(not isOpen):
          clock = self.alpaca.get_clock()
          openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
          currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
          timeToOpen = int((openingTime - currTime) / 60)
          print(str(timeToOpen) + " minutes til market open.")
          time.sleep(60)
          isOpen = self.alpaca.get_clock().is_open
    
    def MA(self,df, n):  #######Carson Created from https://www.quantopian.com/posts/technical-analysis-indicators-without-talib-code
        MA = pd.Series((df['close'].rolling(window=n).mean()), name = 'MA_' + str(n))  
        df = df.join(MA)  
        return df

# Tracks position in list of symbols to download
    def calculateSMAs(self, symbol):
        self.SMA20=0
        self.SMA50=0
        # Calculated trading indicators
        #SMA20df = self.MA(self.dataset[symbol],20)
        #self.SMA20=SMA20df.MA_20.values.tolist()[-1]
        #SMA50df = self.MA(self.dataset[symbol],50)
        #self.SMA50=SMA50df.MA_50.values.tolist()[-1]
        SMA20df = self.MA(self.dataset[symbol],20)
        self.SMA20=SMA20df['MA_20'].values[-1]
        SMA50df = self.MA(self.dataset[symbol],50)
        self.SMA50=SMA50df['MA_50'].values[-1]
    
    # Calculates the trading signals
    def buyselldecision(self, symbol):
        price = self.alpaca.get_last_trade(symbol).price
        print("tried" + symbol + "| P"+str(price)+ "| SMA"+str(self.SMA20))
        #print('price '+str(price))
        #print('SMA20 '+str(self.SMA20))
        try:
            openPosition = int(self.alpaca.get_position(symbol).qty)
        except:
            # No position exists
            openPosition = 0
        print('current holding:'+ str(openPosition))
        if price > self.SMA20:
            # Opens new position if one does not exist
            if openPosition <= 1:
                cashBalance = self.alpaca.get_account().cash
                targetPositionSize = ((float(cashBalance) * self.positionSizing) // price)-1 # Calculates required position size

                returned = self.alpaca.submit_order(symbol,targetPositionSize,"buy","market","gtc") # Market order to open position
                print(returned)

        else:
            # Closes position if SMA20 is below SMA50
            if openPosition > 1:
                try:
                    SellMinusOne = openPosition -1
                    returned = self.alpaca.submit_order(symbol,SellMinusOne,"sell","market","gtc") # Market order to fully close position
                    print(returned)
                except: 
                    print('maintaining 1 share')
    
    
SMAS = SMAStrat()
SMAS.run()

Waiting for market to open...
Market opened.
triedSPY| P338.69| SMA330.3222500000001
current holding:37
triedMSFT| P210.29| SMA208.25312500000024
current holding:32
triedAAPL| P462.38| SMA423.5117000000002
current holding:21
triedNFLX| P492.69| SMA488.5211249999999
current holding:12
triedMRK| P84.595| SMA81.00931499999999
current holding:94
triedPEP| P138| SMA137.05249999999995
current holding:56
triedDG| P199.61| SMA193.96650000000005
current holding:45
triedADP| P140.48| SMA139.47949999999997
current holding:37
triedSPY| P338.8| SMA329.1057499999999
current holding:37
triedMSFT| P210.24| SMA208.24962500000024
current holding:32
triedAAPL| P463.12| SMA416.6187000000001
current holding:21
triedNFLX| P492.57| SMA489.92577500000027
current holding:12
triedMRK| P84.73| SMA81.01881499999999
current holding:94
triedPEP| P138.18| SMA137.06299999999996
current holding:56
triedDG| P199.7| SMA193.46250000000003
current holding:45
triedADP| P140.42| SMA140.292
current holding:37
triedSPY| P338.

triedMRK| P84.43| SMA80.56381500000002
current holding:94
triedPEP| P138.005| SMA136.65574999999993
current holding:56
triedDG| P198.06| SMA193.38325000000003
current holding:45
triedADP| P139.865| SMA140.26500000000001
current holding:1
triedSPY| P338.5| SMA330.31675000000007
current holding:37
triedMSFT| P211.54| SMA208.31062500000021
current holding:32
triedAAPL| P462.13| SMA423.4867000000002
current holding:21
triedNFLX| P491.54| SMA488.4785
current holding:12
triedMRK| P84.56| SMA81.00981499999997
current holding:94
triedPEP| P138.12| SMA137.06099999999998
current holding:56
triedDG| P197.9| SMA193.88500000000005
current holding:45
triedADP| P139.98| SMA139.45499999999998
current holding:1
Order({   'asset_class': 'us_equity',
    'asset_id': '6ca2e51d-21df-4ff0-a134-abfdbe084142',
    'canceled_at': None,
    'client_order_id': '049c305c-fab4-415c-a9c2-82d2989d6256',
    'created_at': '2020-08-18T20:03:44.37779Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_a

In [1]:
import alpaca_trade_api as tradeapi
from datetime import datetime
import pandas as pd

APCA_API_KEY_ID = 'PKTVWYF8T2T1UEUJVA5T'
APCA_API_SECRET_KEY = '2euAseS6hCblaZQRAE0odvz3/zuLttrz85ba8vyr'
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'

api = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, api_version='v2')

activities=api.get_activities()

timeList = []
qtyList = []
symbolList = []
sideList = []

# Reads, formats and stores the new bars
for i in range(len(activities)):
    timeList.append(datetime.fromisoformat(str((activities[i].transaction_time))))
    #timeList.append(activities[i].transaction_time)
    symbolList.append(activities[i].symbol)
    sideList.append(activities[i].side)
    qtyList.append(activities[i].qty)
df = pd.DataFrame(list(zip(timeList,symbolList,sideList, qtyList)), 
               columns =['time','symbol','side', 'qty']) 
df 


,time,symbol,side,qty
0,2020-08-10 16:55:40.449098+00:00,MSFT,buy,37
1,2020-08-10 14:52:20.629530+00:00,MSFT,sell,35
2,2020-08-10 14:42:13.444636+00:00,PEP,sell,45
3,2020-08-10 13:30:55.356365+00:00,PEP,buy,45
4,2020-08-10 13:30:54.301990+00:00,NFLX,sell,13
5,2020-08-10 19:17:58.406186+00:00,PEP,sell,58
6,2020-08-10 18:57:43.155430+00:00,PEP,buy,40
7,2020-08-10 18:57:42.982383+00:00,PEP,buy,18
8,2020-08-10 18:16:57.447342+00:00,PEP,sell,58
9,2020-08-10 18:16:51.454943+00:00,MSFT,sell,33


In [1]:
#import sys
#sys.path.append('path to the package you're wanting to import)
from Sentiment import performanceemail

ModuleNotFoundError: No module named 'Sentiment'

In [ ]:
##Add some sort of logic that requires two consequitive 10 minute bars to be above or below the sma before trading?
## build logic that it only trades w/ crossover, not if point in time it's above or below
## how to incorporate after market trding 
##why are sma's on tradingview not step functions?? Look at the source code for 20SMA in trading view
##do i need to build in day trader rules? What if i leave one share at all times
##See if we can short in the live account and add short logic in. get list of easy to borrow stocks
## Could use the stock screener in tradingview to get a list of securities to trade
##how do i get around the day trade rule? have the sma use closing and then 15 min or something for the t-0?